<a href="https://colab.research.google.com/github/ayushaditya1/WaterQuality-prediction/blob/main/water_predection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
import pandas as pd # data manipulation
import numpy as np # numerical python - linear algebra

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [12]:
# load the dataset
df = pd.read_csv('/content/water prediction.csv', sep=';')

In [5]:
df

,id;date;NH4;BSK5;Suspended;O2;NO3;NO2;SO4;PO4;CL
0,1;17.02.2000;0.33;2.77;12;12.3;9.5;0.057;154;0...
1,1;11.05.2000;0.044;3;51.6;14.61;17.75;0.034;35...
2,1;11.09.2000;0.032;2.1;24.5;9.87;13.8;0.173;41...
3,1;13.12.2000;0.17;2.23;35.6;12.4;17.13;0.099;2...
4,1;02.03.2001;0;3.03;48.8;14.69;10;0.065;281.6;...
...,...
2856,22;06.10.2020;0.046;2.69;3.6;8.28;3.8;0.038;16...
2857,22;27.10.2020;0;1.52;0.5;11.26;0.56;0.031;147....
2858,22;03.12.2020;0.034;0.29;0.8;11.09;2.58;0.042;...
2859,22;12.01.2021;0;2.1;0;14.31;3.94;0.034;121.6;0...


In [6]:
df.info() # dataset info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2861 entries, 0 to 2860
Data columns (total 1 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   id;date;NH4;BSK5;Suspended;O2;NO3;NO2;SO4;PO4;CL  2861 non-null   object
dtypes: object(1)
memory usage: 22.5+ KB


In [7]:
# rows and cols
df.shape

(2861, 1)

In [8]:
# Statistics of the data
df.describe().T

,count,unique,top,freq
id;date;NH4;BSK5;Suspended;O2;NO3;NO2;SO4;PO4;CL,2861,2861,22;10.02.2021;0;1.78;0;14.3;6.3;0.033;134.4;0....,1


In [9]:
# Missing values
df.isnull().sum()

,0
id;date;NH4;BSK5;Suspended;O2;NO3;NO2;SO4;PO4;CL,0


In [13]:
# date is in object - date format
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,2000-02-17,0.330,2.77,12.0,12.30,9.50,0.057,154.00,0.454,289.50
1,1,2000-05-11,0.044,3.00,51.6,14.61,17.75,0.034,352.00,0.090,1792.00
2,1,2000-09-11,0.032,2.10,24.5,9.87,13.80,0.173,416.00,0.200,2509.00
3,1,2000-12-13,0.170,2.23,35.6,12.40,17.13,0.099,275.20,0.377,1264.00
4,1,2001-03-02,0.000,3.03,48.8,14.69,10.00,0.065,281.60,0.134,1462.00
...,...,...,...,...,...,...,...,...,...,...,...
2856,22,2020-10-06,0.046,2.69,3.6,8.28,3.80,0.038,160.00,0.726,77.85
2857,22,2020-10-27,0.000,1.52,0.5,11.26,0.56,0.031,147.20,0.634,71.95
2858,22,2020-12-03,0.034,0.29,0.8,11.09,2.58,0.042,209.92,0.484,61.17
2859,22,2021-01-12,0.000,2.10,0.0,14.31,3.94,0.034,121.60,0.424,63.49


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2861 entries, 0 to 2860
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         2861 non-null   int64         
 1   date       2861 non-null   datetime64[ns]
 2   NH4        2858 non-null   float64       
 3   BSK5       2860 non-null   float64       
 4   Suspended  2845 non-null   float64       
 5   O2         2858 non-null   float64       
 6   NO3        2860 non-null   float64       
 7   NO2        2858 non-null   float64       
 8   SO4        2812 non-null   float64       
 9   PO4        2833 non-null   float64       
 10  CL         2812 non-null   float64       
dtypes: datetime64[ns](1), float64(9), int64(1)
memory usage: 246.0 KB


In [15]:
df = df.sort_values(by=['id', 'date'])
df.head()

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,2000-02-17,0.330,2.77,12.0,12.30,9.50,0.057,154.0,0.454,289.5
1,1,2000-05-11,0.044,3.00,51.6,14.61,17.75,0.034,352.0,0.090,1792.0
2,1,2000-09-11,0.032,2.10,24.5,9.87,13.80,0.173,416.0,0.200,2509.0
3,1,2000-12-13,0.170,2.23,35.6,12.40,17.13,0.099,275.2,0.377,1264.0
4,1,2001-03-02,0.000,3.03,48.8,14.69,10.00,0.065,281.6,0.134,1462.0


In [16]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

In [17]:
df.head()

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL,year,month
0,1,2000-02-17,0.330,2.77,12.0,12.30,9.50,0.057,154.0,0.454,289.5,2000,2
1,1,2000-05-11,0.044,3.00,51.6,14.61,17.75,0.034,352.0,0.090,1792.0,2000,5
2,1,2000-09-11,0.032,2.10,24.5,9.87,13.80,0.173,416.0,0.200,2509.0,2000,9
3,1,2000-12-13,0.170,2.23,35.6,12.40,17.13,0.099,275.2,0.377,1264.0,2000,12
4,1,2001-03-02,0.000,3.03,48.8,14.69,10.00,0.065,281.6,0.134,1462.0,2001,3


In [18]:
df.columns

Index(['id', 'date', 'NH4', 'BSK5', 'Suspended', 'O2', 'NO3', 'NO2', 'SO4',
       'PO4', 'CL', 'year', 'month'],
      dtype='object')

In [19]:
pollutants = ['O2', 'NO3', 'NO2', 'SO4',
       'PO4', 'CL']

In [21]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Select only numerical columns for imputation and scaling
numerical_cols = df.select_dtypes(include=np.number).columns
df_numerical = df[numerical_cols]

imputer = SimpleImputer(strategy='mean')
df_imputed = pd.DataFrame(imputer.fit_transform(df_numerical), columns=numerical_cols)

scaler = StandardScaler()
X = scaler.fit_transform(df_imputed) # Removed .drop('Potability', axis=1) and applied to the imputed numerical data
# Assuming you have a target variable 'y'. If not, you'll need to define it.
# For example, if you want to predict 'O2', you would set y = df_imputed['O2']
# y = df_imputed['Potability'] # This line is commented out as 'Potability' does not exist

In [23]:
from sklearn.model_selection import train_test_split

# Define the target variables
y = df_imputed[pollutants]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [25]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

model = MultiOutputRegressor(RandomForestRegressor(random_state=42))
model.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

In [27]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test)

# Evaluate the model for each target variable
for i, pollutant in enumerate(pollutants):
    mse = mean_squared_error(y_test.iloc[:, i], y_pred[:, i])
    r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
    print(f"Metrics for {pollutant}:")
    print(f"  Mean Squared Error (MSE): {mse:.4f}")
    print(f"  R-squared (R2): {r2:.4f}")
    print("-" * 20)

Metrics for O2:
  Mean Squared Error (MSE): 0.8656
  R-squared (R2): 0.9558
--------------------
Metrics for NO3:
  Mean Squared Error (MSE): 0.1579
  R-squared (R2): 0.9948
--------------------
Metrics for NO2:
  Mean Squared Error (MSE): 2.8443
  R-squared (R2): -1.9717
--------------------
Metrics for SO4:
  Mean Squared Error (MSE): 584.1726
  R-squared (R2): 0.8545
--------------------
Metrics for PO4:
  Mean Squared Error (MSE): 0.0104
  R-squared (R2): 0.9876
--------------------
Metrics for CL:
  Mean Squared Error (MSE): 76.6692
  R-squared (R2): 0.9993
--------------------


In [30]:
# Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import xgboost as xgb

# Load Dataset
df = pd.read_csv('/content/water prediction.csv', sep=';')

# Handle Missing Values
imputer = SimpleImputer(strategy='mean')
df_imputed = pd.DataFrame(imputer.fit_transform(df.select_dtypes(include=np.number)), columns=df.select_dtypes(include=np.number).columns)


# Features and Target
# Assuming you are trying to predict the pollutants as in the previous successful run
pollutants = ['O2', 'NO3', 'NO2', 'SO4', 'PO4', 'CL']
X = df_imputed.drop(columns=pollutants)
y = df_imputed[pollutants]


# Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-Test Split
# Note: Stratify is typically used for classification tasks to maintain class distribution.
# Since this is a regression task, stratify is not appropriate.
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Handle Imbalanced Classes with SMOTE
# Note: SMOTE is used for balancing classes in classification tasks.
# It is not appropriate for regression tasks. I will comment this out.
# smote = SMOTE(random_state=42)
# X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Model: XGBoost
# Note: XGBoostClassifier is for classification. For regression, use XGBoostRegressor.
# Also, MultiOutputRegressor is needed for multiple target variables.
from sklearn.multioutput import MultiOutputRegressor
model = MultiOutputRegressor(xgb.XGBRegressor(random_state=42))
model.fit(X_train, y_train)

# Evaluate on Test Set
# Note: Classification metrics are not suitable for regression.
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test)

for i, pollutant in enumerate(pollutants):
    mse = mean_squared_error(y_test.iloc[:, i], y_pred[:, i])
    r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
    print(f"Metrics for {pollutant}:")
    print(f"  Mean Squared Error (MSE): {mse:.4f}")
    print(f"  R-squared (R2): {r2:.4f}")
    print("-" * 20)

# Optional: Cross-Validation F1 Score
# Note: F1 score is for classification. For regression, use a suitable regression score.
# Cross-validation can still be useful for regression models.
from sklearn.model_selection import cross_val_score
# For example, using negative mean squared error as the scoring metric
# scores = cross_val_score(model, X_scaled, y, cv=5, scoring='neg_mean_squared_error')
# print("Average Cross-Validated MSE:", -scores.mean())

Metrics for O2:
  Mean Squared Error (MSE): 19.8380
  R-squared (R2): -0.0125
--------------------
Metrics for NO3:
  Mean Squared Error (MSE): 45.5361
  R-squared (R2): -0.4937
--------------------
Metrics for NO2:
  Mean Squared Error (MSE): 0.4620
  R-squared (R2): 0.5174
--------------------
Metrics for SO4:
  Mean Squared Error (MSE): 1208.3379
  R-squared (R2): 0.6991
--------------------
Metrics for PO4:
  Mean Squared Error (MSE): 0.8976
  R-squared (R2): -0.0670
--------------------
Metrics for CL:
  Mean Squared Error (MSE): 24085.8503
  R-squared (R2): 0.7920
--------------------
